In [1]:
import cv2
import mediapipe as mp
import time  # for identify the current time 

In [ ]:
# connecting with default camera 0
cap=cv2.VideoCapture(0)

#### Showing frames on window 

In [ ]:
# untile sucess ( in this case infinite untile we close the terminal
while True:
    # reading image and suceess status from camera using read()
    success, img = cap.read()
    # displaying that img(frame ) on window using opencv imshow()
    cv2.imshow("Image capture", img)
    # 1 frame generate after 1milliosecond
    cv2.waitKey(1)

#### Creating mediapipe objects and processing each frame using them, and generating each frame hands landmarks positions

In [ ]:
# this contains all information about hands , and its detection 
mp_hands = mp.solutions.hands
# object of Hands class , default variables like no of hands =2 , etc
hands = mp_hands.Hands()

In [ ]:
while True:
    success, img = cap.read()

    # mediapipe works only on RGB color image
    # converting into rgb
    img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    # process() used to detect hands in an each frame
    # now, result contains all hands landmarks details
    # NOTE :- each image(frame) landmarks data are in List() formate inside results.
    # and these results are of similiar to dictionary type ( mediapipe object )
    results = hands.process(img_rgb)     # result type is List( mediapipe objects similiar to dict in it) = [ {}, {}, ... , {} ] 
    #### check this out
    #print(results.multi_hand_landmarks)
    #print(type(results.multi_hand_landmarks))
    
    # if hand detecting 
    if results.multi_hand_landmarks:
        # accessing each individual landmark points one by one from result LIST()
        # "hand_landmarks" is of type :- <class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmarkList'> 
        # similiar to dictionary objects
        # NOTE :- at each iteration "hands_landmarks" added landmark details into its dictionary type object
        for hands_landmarks in results.multi_hand_landmarks:
            ##### check this out
            #print(hands_landmarks)
            #print(type(hands_landmarks))

            #### check this 
            #<class 'google.protobuf.internal.containers.RepeatedCompositeFieldContainer'> = list()
            # all "hands_landmarks" items into list() type formate, so, easy to identify indexes of each landmarks
            #print(hands_landmarks.landmark)
            #print(type(hands_landmarks.landmark))

            # "id" is of integer type
            # "lm" is <class 'mediapipe.framework.formats.landmark_pb2.NormalizedLandmark'> type or similiar to dictionary
            for id,lm in enumerate(hands_landmarks.landmark):
                #print(id,lm)
                #print(type(id),type(lm))
                
    cv2.imshow("Image capture", img)

    # capture only one frame untile we can't response or create event 
    cv2.waitKey(0)

#### Converting landmarks positions values into pixels formate and displaying "filled circle" on each landmark points:-
USING cv2.circle()

In [ ]:
while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)    
    
    if results.multi_hand_landmarks:
        for hands_landmarks in results.multi_hand_landmarks:
            for id,lm in enumerate(hands_landmarks.landmark):

                # img.shape = (height, width, channel(RGB) )
                height, width, channel = img.shape
                # converting fractional x and y axis points to pixels on the basis of image shape
                # 'cx' and 'cy' are the positions of landmark from the central top left of the image
                cx, cy = int(lm.x*width), int(lm.y*height)
                # displaying each frame's 21 landmarks along with their 'id' number
                print(id, cx, cy)
                # calling cv2 method circle()
                # img :- actual image
                # (cx, cy) :- landmarks positions on the image
                #5 :- radius of the circle
                # (255,0,255) color of the circle
                # cv2.FILLED :- circle is filled with boder color
                cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED
                
    cv2.imshow("Image capture", img)
    cv2.waitKey(1)

#### Displaying landmark points and their connecting line with the help of mediapipe drawing utilities:-
#### And displaying frame-per-second on the camera window using cv2 method
using mediapipe drawing utilities

In [ ]:
# mediapipe drawing utilities
mp_draw = mp.solutions.drawing_utils

# for calculating frame per second
current_time=0
previews_time=0

while True:
    success, img = cap.read()
    img_rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)    
    
    if results.multi_hand_landmarks:
        for hands_landmarks in results.multi_hand_landmarks:
            # displaying landmarks and connecting lines on frame using mediapipe utils
            # mp_hands.HAND_CONNECTIONS :- used to draw lines between landmark points
            mp_draw.draw_landmarks(img,hands_landmarks,mp_hands.HAND_CONNECTIONS)

    # time.time() returns time in second from 1970 ( first epoch )
    # to get your device time epoch , 
    #obj = time.gmtime(0) 
    #epoch = time.asctime(obj) 
    current_time = time.time()
    # time required to generate one frame
    frame_per_second = 1/(current_time - previews_time)
    previews_time = current_time

    # for displaying frame per second on window
    # frame_per_second :- firstly converting it from fraction to intger , then into string
    # 10,60 is the text display position
    #  cv2.FONT_HERSHEY_PLAIN , font style of the text
    # 2 is the scaler ( higher means larger size )
    # 3 :- thickness of the text
    cv2.putText(img,str(int(frame_per_second)), ( 10, 60), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 3  )
    
    cv2.imshow("Image capture", img)
    cv2.waitKey(1)